In [8]:
import torch
import torch.nn as nn
import instruction_test_data as itd
import numpy as np
from card_embedding import FilterEmbedding, SharedEmbeddingHolder, NormalizedLinear, AttackDataEmbedding, DiscardDataEmbedding, CardAmountDataEmbedding, ReturnToDeckTypeDataEmbedding, PlayerTargetDataEmbedding
import nesting
from multi_head_attention import MultiHeadAttention

d=8
shared_embedding_holder = SharedEmbeddingHolder(d, device='cuda')

In [9]:
instruction_types, instruction_indices, instruction_data_types, instruction_data_type_indices, instruction_data, instruction_data_indices = nesting.flatten_instructions(itd.instructions_batch, device='cuda')

In [10]:
instruction_data

([tensor([ 0, 10], device='cuda:0'), tensor([ 0, 70], device='cuda:0')],
 [tensor([2], device='cuda:0'),
  tensor([1], device='cuda:0'),
  tensor([1], device='cuda:0')],
 [tensor([2, 2, 8], device='cuda:0'),
  tensor([0, 1, 7], device='cuda:0'),
  tensor([-1, -1, 10], device='cuda:0'),
  tensor([1, 1, 5], device='cuda:0'),
  tensor([-1, -1, 10], device='cuda:0'),
  tensor([2, 2, 7], device='cuda:0'),
  tensor([1, 1, 4], device='cuda:0'),
  tensor([-1, -1, 10], device='cuda:0')],
 [tensor([0, 9], device='cuda:0')],
 [{'Operands': [],
   'LogicalOperator': 0,
   'IsLeaf': True,
   'Condition': {'Field': 2, 'Operation': 0, 'Value': -1}},
  {'Operands': [],
   'LogicalOperator': 0,
   'IsLeaf': True,
   'Condition': {'Field': 3, 'Operation': 1, 'Value': 1}},
  {'Operands': [{'Operands': [],
     'LogicalOperator': 0,
     'IsLeaf': True,
     'Condition': {'Field': 3, 'Operation': 1, 'Value': 1}},
    {'Operands': [],
     'LogicalOperator': 0,
     'IsLeaf': True,
     'Condition': {'Fiel

In [11]:
instruction_data_indices

([(2, 0, 0), (3, 0, 0)],
 [(0, 1, 0), (0, 6, 0), (1, 3, 0)],
 [(0, 0, 0),
  (0, 2, 0),
  (0, 4, 0),
  (1, 0, 0),
  (1, 2, 0),
  (4, 0, 0),
  (4, 1, 0),
  (4, 2, 0)],
 [(4, 3, 0)],
 [(0, 0, 1), (0, 2, 1), (1, 0, 1), (4, 1, 1)],
 [(0, 5, 0)])

In [12]:
attack_data= torch.stack(instruction_data[0])
print(attack_data)
print(attack_data[:,0])
print(attack_data[:,1])
attack_data_embedding = AttackDataEmbedding(8, device='cuda')
print(attack_data_embedding(attack_data))
print(attack_data_embedding(attack_data).size())

tensor([[ 0, 10],
        [ 0, 70]], device='cuda:0')
tensor([0, 0], device='cuda:0')
tensor([10, 70], device='cuda:0')
tensor([[ 0.8276, -2.0796, -0.4522,  1.1406,  0.7865,  0.3309,  0.1415, -0.3769],
        [ 0.9722, -1.9412, -0.5720,  1.2607,  0.8419,  0.4174,  0.1827, -0.3138]],
       device='cuda:0', grad_fn=<AddBackward0>)
torch.Size([2, 8])


In [13]:
discard_data= torch.tensor(instruction_data[1], device='cuda')
print(discard_data)
discard_data_embedding = DiscardDataEmbedding(8, device='cuda')
print(discard_data_embedding(discard_data))
print(discard_data_embedding(discard_data).size())

tensor([2, 1, 1], device='cuda:0')
tensor([[ 1.1419,  0.4338, -1.4672,  0.1070, -2.7960, -0.4082, -0.1120, -0.7619],
        [-0.7828, -3.1895,  0.6799, -1.1064,  0.8218, -0.8566,  0.6500, -0.7707],
        [-0.7828, -3.1895,  0.6799, -1.1064,  0.8218, -0.8566,  0.6500, -0.7707]],
       device='cuda:0', grad_fn=<EmbeddingBackward0>)
torch.Size([3, 8])


In [14]:
card_amount_data= torch.stack(instruction_data[2])
print(card_amount_data)
print(card_amount_data[:,0:2])
print(card_amount_data[:,2])
card_amount_data_embedding = CardAmountDataEmbedding(shared_embedding_holder, d, device='cuda')
print(card_amount_data_embedding(card_amount_data))
print(card_amount_data_embedding(card_amount_data).size())


tensor([[ 2,  2,  8],
        [ 0,  1,  7],
        [-1, -1, 10],
        [ 1,  1,  5],
        [-1, -1, 10],
        [ 2,  2,  7],
        [ 1,  1,  4],
        [-1, -1, 10]], device='cuda:0')
tensor([[ 2,  2],
        [ 0,  1],
        [-1, -1],
        [ 1,  1],
        [-1, -1],
        [ 2,  2],
        [ 1,  1],
        [-1, -1]], device='cuda:0')
tensor([ 8,  7, 10,  5, 10,  7,  4, 10], device='cuda:0')
tensor([[ 1.2796,  0.2978, -1.6028,  1.4369, -1.7501, -0.1528,  1.6623, -2.3437],
        [-0.8354,  1.1387,  0.1663, -0.3706, -0.6742,  0.5042,  0.4181, -0.0689],
        [ 1.4766,  0.2806,  2.9991, -1.9369,  0.8828, -1.3831, -0.4896,  0.5260],
        [-1.3097, -0.7277,  0.8864,  1.4407, -1.2723, -0.2631,  0.3757,  0.2811],
        [ 1.4766,  0.2806,  2.9991, -1.9369,  0.8828, -1.3831, -0.4896,  0.5260],
        [-0.8348,  1.1400,  0.1638, -0.3697, -0.6754,  0.4995,  0.4165, -0.0685],
        [-0.5445,  0.3417,  0.8721,  0.6408,  0.8297, -0.0892,  0.0074,  0.0819],
        [ 1.

In [15]:
return_to_deck_type_data = torch.stack(instruction_data[3])
print(return_to_deck_type_data)
print(return_to_deck_type_data[:, 0])
print(return_to_deck_type_data[:, 1])
return_to_deck_type_data_embedding = ReturnToDeckTypeDataEmbedding(
    shared_embedding_holder, d, device='cuda'
)
embedded_return_to_deck_type_data = return_to_deck_type_data_embedding(return_to_deck_type_data)
print(embedded_return_to_deck_type_data)
print(embedded_return_to_deck_type_data.size())

tensor([[0, 9]], device='cuda:0')
tensor([0], device='cuda:0')
tensor([9], device='cuda:0')
tensor([[-0.6631,  0.8867,  0.4409,  1.2479, -0.8346, -0.4733, -1.2001,  0.0907]],
       device='cuda:0', grad_fn=<AddBackward0>)
torch.Size([1, 8])


In [16]:
filter_data = instruction_data[4]
print(filter_data)
filter_embedding = FilterEmbedding(shared_embedding_holder, d, device='cuda')
embedded_filter_data = filter_embedding.forward_v2(filter_data)
print(embedded_filter_data)
print(embedded_filter_data.size())


[{'Operands': [], 'LogicalOperator': 0, 'IsLeaf': True, 'Condition': {'Field': 2, 'Operation': 0, 'Value': -1}}, {'Operands': [], 'LogicalOperator': 0, 'IsLeaf': True, 'Condition': {'Field': 3, 'Operation': 1, 'Value': 1}}, {'Operands': [{'Operands': [], 'LogicalOperator': 0, 'IsLeaf': True, 'Condition': {'Field': 3, 'Operation': 1, 'Value': 1}}, {'Operands': [], 'LogicalOperator': 0, 'IsLeaf': True, 'Condition': {'Field': 4, 'Operation': 1, 'Value': 8}}], 'LogicalOperator': 2, 'IsLeaf': False, 'Condition': None}, {'Operands': [], 'LogicalOperator': 0, 'IsLeaf': True, 'Condition': {'Field': 1, 'Operation': 0, 'Value': -1}}]
tensor([[-0.5648, -1.2544, -0.6870, -1.0572, -0.7275,  1.1842, -1.5258, -0.6870],
        [ 0.5402,  0.8311,  1.0681,  1.2784, -2.4457,  1.9160, -2.0477, -1.8444],
        [ 3.1468,  0.9788, -4.7799, -0.0609, -1.0961,  3.9680, -3.7570, -3.6892],
        [-0.7005,  0.6115,  0.5130,  0.7611,  0.7567,  0.3726,  0.0793, -0.5155]],
       device='cuda:0', grad_fn=<StackB

In [18]:
player_target_data = torch.tensor(instruction_data[5], device='cuda')
print(player_target_data)
player_target_data_embedding = PlayerTargetDataEmbedding(shared_embedding_holder, d, device='cuda')
embedded_player_target_data = player_target_data_embedding(player_target_data)
print(embedded_player_target_data)
print(embedded_player_target_data.size())

tensor([0], device='cuda:0')
tensor([[-2.4971, -0.3819,  0.3101,  0.2596,  0.0289,  1.5154, -1.0115,  0.8181]],
       device='cuda:0', grad_fn=<EmbeddingBackward0>)
torch.Size([1, 8])


In [31]:
from itertools import chain


class InstructionEmbedding(nn.Module):
    def __init__(
        self,
        shared_embedding_holder: SharedEmbeddingHolder,
        dimension_out: int,
        device=None,
        dtype=None,
    ):
        self.factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.dimension_out = dimension_out
        self.attack_data_embedding = AttackDataEmbedding(
            dimension_out, **self.factory_kwargs
        )
        self.discard_data_embedding = DiscardDataEmbedding(
            dimension_out, **self.factory_kwargs
        )
        self.card_amount_data_embedding = CardAmountDataEmbedding(
            shared_embedding_holder, dimension_out, **self.factory_kwargs
        )
        self.return_to_deck_type_data_embedding = ReturnToDeckTypeDataEmbedding(
            shared_embedding_holder, dimension_out, **self.factory_kwargs
        )
        self.filter_embedding = FilterEmbedding(
            shared_embedding_holder, dimension_out, **self.factory_kwargs
        )
        self.player_target_data_embedding = PlayerTargetDataEmbedding(
            shared_embedding_holder, dimension_out, **self.factory_kwargs
        )
        self.instruction_type_embedding = nn.Embedding(
            8, dimension_out, padding_idx=0, **self.factory_kwargs
        )
        self.instruction_data_type_embedding = nn.Embedding(
            6, dimension_out, padding_idx=0, **self.factory_kwargs
        )
        self.data_multi_head_attention = MultiHeadAttention(
            dimension_out, dimension_out, dimension_out, max(dimension_out//16, 4), 4, **self.factory_kwargs
        )

    def forward(self, instructions_batch: list[list[dict]]) -> torch.Tensor:
        (
            instruction_types,
            instruction_indices,
            instruction_data_types,
            instruction_data_type_indices,
            instruction_data,
            instruction_data_indices,
        ) = nesting.flatten_instructions(instructions_batch, **self.factory_kwargs)
        instruction_type_embeddings = self.instruction_type_embedding(instruction_types)
        instruction_data_type_embeddings = self.instruction_data_type_embedding(
            instruction_data_types
        )
        attack_data_embeddings = self.attack_data_embedding(
            torch.stack(instruction_data[0])
        )
        discard_data_embeddings = self.discard_data_embedding(
            torch.tensor(instruction_data[1], **self.factory_kwargs)
        )
        card_amount_data_embeddings = self.card_amount_data_embedding(
            torch.stack(instruction_data[2])
        )
        return_to_deck_type_data_embeddings = self.return_to_deck_type_data_embedding(
            torch.stack(instruction_data[3])
        )
        filter_embeddings = self.filter_embedding(instruction_data[4])
        player_target_data_embeddings = self.player_target_data_embedding(
            torch.tensor(instruction_data[5], **self.factory_kwargs)
        )

        sorted_data = self.sort_tensors_with_respect_to_index(
            (
                attack_data_embeddings,
                discard_data_embeddings,
                card_amount_data_embeddings,
                return_to_deck_type_data_embeddings,
                filter_embeddings,
                player_target_data_embeddings,
            ),
            instruction_data_indices,
        )
        data_tensors = sorted_data + instruction_data_type_embeddings

        instruction_embeddings = self.embed_instruction_data(instruction_indices, instruction_data_type_indices, instruction_type_embeddings, data_tensors)

        return (
            instruction_embeddings,
            instruction_indices,
        )

    def sort_tensors_with_respect_to_index(self, tensors, indices):
        return torch.stack(
            [
                tensor
                for _, tensor in sorted(
                    zip(
                        chain.from_iterable(indices),
                        chain.from_iterable(tensors),
                    ),
                    key=lambda pair: pair[0],
                )
            ]
        )
    
    def embed_instruction_data(self, instruction_indices, instruction_data_type_indices, instruction_type_embeddings, data_tensors):
        query_list = []
        for i, instruction_index in enumerate(instruction_indices):
            unbatched_query = torch.cat(
                [
                    instruction_type_embeddings[i].unsqueeze(0),
                    data_tensors[
                        (
                            instruction_data_type_indices[:, 0:2] == instruction_index
                        ).sum(1)
                        == 2
                    ],
                ]
            )
            query_list.append(unbatched_query)
        query_tensor = torch.nested.nested_tensor(query_list, layout=torch.jagged)
        return (
            query_tensor
            + self.data_multi_head_attention(query_tensor, query_tensor, query_tensor)
        ).sum(1)


instruction_embedding = InstructionEmbedding(shared_embedding_holder, d, device="cuda")
(
    instruction_embeddings,
            instruction_indices,
) = instruction_embedding(itd.instructions_batch)
print(instruction_embeddings.size())
print(instruction_indices)

torch.Size([17, 8])
tensor([[0, 0],
        [0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [0, 5],
        [0, 6],
        [1, 0],
        [1, 1],
        [1, 2],
        [1, 3],
        [2, 0],
        [3, 0],
        [4, 0],
        [4, 1],
        [4, 2],
        [4, 3]], device='cuda:0')
